# Ai-core project 1 - Scraping Nutritional Values of Food Products Available in the UK market 2021
## Functions include:
        F-1 load_and_except_cookies
        F-2 get_categories
        F-3 get_subcategories
        F-4 get_level_3_categories
        F-5 get_products_links_for_all_articles_method2
        F-6 get_all_products_data
        
       

## Import all dependencies

In [6]:
from selenium import webdriver
from time import sleep
import requests
from lxml import etree, html 
import pandas as pd
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Instantiate selenium webdriver

In [197]:
#instantiate a driver
driver = webdriver.Chrome('chrome_driver/chromedriver') 
URL = "https://www.waitrose.com/ecom/shop/browse/groceries"
driver.get(URL)

## Load and except cookies

In [77]:
def load_and_accept_cookies():
        
    driver = webdriver.Chrome('chrome_driver/chromedriver') 
    URL = "https://www.waitrose.com/ecom/shop/search?ct=Breakfast&searchTerm={breakfast}&sortBy=MOST_POPULAR"
    driver.get(URL)
    accept_cookies = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div/section/div[2]/button[1]')

    
    for button in accept_cookies:
        if button.text == "Yes, allow all":
            relevant_button = button

    relevant_button.click()
    return driver
load_and_accept_cookies()

<selenium.webdriver.chrome.webdriver.WebDriver (session="256b90bb537f90c7094a84ebef06d5ca")>

#### Get all categories, subcategories and subcategories within the subcategories

In [163]:
#getting all the subcategories and sub-subcategories
def get_categories():
    '''
    defining category levels
    '''
    categories = level1_cat
    subcategories = level2_cat
    sub_subcategories = level3_cat

categories = ['fresh_and_chilled', 'bakery', 'frozen','food_cupboard']
subcategoryDict = {'fresh_and_chilled': [] , 'bakery' : [], 'frozen' : [], 'food_cupboard' :[] }
 #for item, xpath in XPATH_prod_data.items():
#create empty lists
fresh_and_chilled = []
bakery = []
frozen = []
food_cupboard =[]
#loop through four items in categories list
def get_subcategories():
    '''
    This function gets all the links to the subcategory items and append them in a dictionery
    '''
    for category in categories:
        driver.get(f'https://www.waitrose.com/ecom/shop/browse/groceries/{category}')
        #for clicking 'Show all button
        driver.find_elements_by_xpath("//*[@id='main']/div[3]/header/div[4]/nav/div/div/div/div/div[2]/div/div/button")
        #driver.find_element(By.XPATH, '//button[text()="Show all"]')

        element = driver.find_elements_by_xpath("//*[@id='subcategoryList']/li/a")
        for elem in element:
            result = elem.get_attribute("href")
            if category == 'fresh_and_chilled':
                subcategoryDict['fresh_and_chilled'].append(result) 
                fresh_and_chilled.append(result) 
            elif category == 'bakery' :
                subcategoryDict['bakery'].append(result) 
                bakery.append(result) 
            elif category == 'frozen':
                subcategoryDict['frozen'].append(result) 
                frozen.append(result) 
            else:
                subcategoryDict['food_cupboard'].append(result) 
                food_cupboard.append(result)  
        for key in subcategoryDict:
           for item in subcategoryDict[key]:
               driver.get(item)

    return subcategoryDict
#go to each items in the subcategories under the categories and get the subcategories 

get_subcategories()


def get_level_3_categories():
    level2_cat_links = []
    level3_cat_links = []
    for key in subcategoryDict:
        for item in subcategoryDict[key]:
            level2_cat_links.append(item)
            print(level2_cat_links[:5])
    for i in level2_cat_links:
        driver.get(i)
        sleep(1)
        level3_cat_elements = driver.find_elements_by_xpath("//*[@id='subcategoryList']/li/a") #to get all the items just use "li" instead of li[1], li[2] etc
        for elem in level3_cat_elements:
            level3_cat_product_links = elem.get_attribute("href")
            level3_cat_links.append(level3_cat_product_links)
    return level3_cat_links



9519420


### Get all product links

In [ ]:

def get_products_links_for_all_articles_method2():
    all_product_links_2 = []
# driver.get('https://www.waitrose.com/ecom/shop/browse/groceries/fresh_and_chilled/fresh_meat/roasting_joints')
    for i in level3_cat_links:
        driver.get(i)
        try:
            driver.find_element_by_xpath("//*[@id='tSr']/div/div[2]/button").click()
            sleep(1)

            print("clicked once")
            driver.find_element_by_xpath("//*[@id='tSr']/div/div[2]/button").click()
            sleep(1)
        except:
            pass

        for x in range(1, 100):
            xpath = "/html/body/div[2]/div/div/main/div[5]/div[2]/div/div[1]/article["
            xpath += str(x)
            xpath += "]/div/section[1]/div[2]/a"
            product_elem = driver.find_elements_by_xpath(xpath)
            
            for elem in product_elem:
                results = elem.get_attribute("href")
                print(results)
                all_product_links_2.append(results)
    print(len(all_product_links_2))

return all_product_links_2

      

In [249]:
print(len(all_product_links_2))

12517


#### Remove duplicate products and save all product links to all_product_links_method2.csv file which will be used to collect data

In [264]:

#create a pandas data frame 
df_all_p = pd.DataFrame(all_product_links_2)
# check if theres any duplicate
#remove duplicates
df_all_p_removeduplicate = df_all_p.drop_duplicates()
#save file as csv
df_all_p_removeduplicate.to_csv('/Users/sz742/Desktop/all_product_links_method2.csv',index=False)

In [265]:
df_all_p_removeduplicate.shape

(7512, 1)

### Get all produts data: product name, weight, servings, price, ingredients, waitrose nutritional values and typical nutritional values

In [110]:
def get_all_products_data(url): 
    product_data = {'product_name' : [], 'weight' : [], 'servings':[], 'price' : [], }
    ingredients= {'product_name': []}


    #GET NUTRITION TYPICAL VALUES

    all_prod_link_list2 = []
    with open('/Users/sz742/Ai-core-Project-1-Web_Scraping/all_product_links_method2.csv', newline='') as file:
        for row in csv.reader(file):
            all_prod_link_list2.append(row[0])
    print(all_prod_link_list2[:5])


    nutri_df= pd.DataFrame()


    for ur in all_prod_link_list2[1:]:
        r = requests.get(ur) # make a HTTP GET request to this website
        html_string = r.text 
        soup = BeautifulSoup(html_string, 'lxml')  
        try:
            table_nutrition = soup.find('table', {'class' : 'table table-striped'}).find('tbody')
        except: AttributeError
        pass
        #     table_nutrition = soup.find('table', {'class' : 'table table-striped'}).find('tbody')
        nutrition = {}
        try:
            product_name = soup.find('span', {'class' : 'name___30fwb'}).text
        except: AttributeError
        pass
        # print(product_name)
        for row in table_nutrition.find_all('tr'):
            key = row.find('th').text
            value = row.find('td').text
            nutrition[key] = value
        nutrition.update({'product_name': product_name})
        nutri_df = nutri_df.append(nutrition, ignore_index=True)
        # nutri_df['product_name'] = product_name
        
        print(ur)

    with open('nutrition_typical_values.csv', 'a') as f:
        nutri_df.to_csv(f, mode='a', header=f.tell()==0) 
    print(nutri_df.head())


  

   
        

    

https://www.waitrose.com/ecom/products/essential-small-whole-chicken-without-giblets/895677-706845-706846


Typical values as prepared. Contains 4 Servings
https://www.waitrose.com/ecom/products/waitrose-crackling-pork-boneless-leg-roast/050735-25241-25242


Typical values as sold. Contains 1 Servings
https://www.waitrose.com/ecom/products/essential-british-chicken-large/645021-507910-507911


Typical values as prepared. Contains 7 Servings
https://www.waitrose.com/ecom/products/no1-corn-fed-free-range-whole-chicken/037011-18529-18530


Typical values as prepared. Contains 4 Servings
https://www.waitrose.com/ecom/products/waitrose-2-british-lamb-leg-shanks/097484-49773-49774


Typical values per edible portion
https://www.waitrose.com/ecom/products/duchy-organic-free-range-british-whole-chicken/078250-39829-39830


Typical values as prepared. Contains 4 Servings
https://www.waitrose.com/ecom/products/essential-british-medium-whole-chicken/845051-714666-714667


Typical values as prepar

IndexError: list index out of range

### Get ingredients and product details data

In [ ]:
 ## GET  INGREDIENTS DATA

ingredients_df = pd.DataFrame()
product_data_df = pd.DataFrame()

for ur in all_prod_link_list2[1:]:
    r = requests.get(ur) # make a HTTP GET request to this website
    html_string = r.text 
    soup = BeautifulSoup(html_string, 'lxml')  
    ingredients = {}
    
    try:
        ingre = soup.find(('div', {'id': "ingredients-region"}))

        convert_str = str(ingre.find('div', {'class': 'ingredientsText___Ztpzh'}).text)
        
    except: AttributeError
    pass

    try:
        product_name = soup.find('span', {'class' : 'name___30fwb'}).text
    except:
        pass
    # print(product_name)
    try: 
        key = 'product_name'
        value = convert_str
        ingredients[key] = value
        ingredients.update ({'product_name': product_name})
    except:
        pass
    #append to the dataframe
    ingredients_df = ingredients_df.append(ingredients, ignore_index=True)
## GET  PRODUCT DETAILS DATA
    #get product weight
    try:
        product_weight = soup.find('span', {'class' :'size___2HSwr sizeMessage___3o5Ri'})
        key1 = 'weight'
        for p in product_weight:
            values1 = p
            product_details[key1] = values1
    except:
        pass
        #get product servings
    try:
        product_serving = soup.find('div', {'class' :'nutrition___24-Nx'})
    
        for ser in product_serving.find('p'):
            # key2 = 'servings'
            values2 = ser
            product_details.update({'servings' : values2})
    
    except:
        pass

    #get product price
    try:
        product_price = soup.find('span', {'data-test' :'product-pod-price'})
    
        for s in product_price.find('span'):
            key3 = 'price'
            values3 = s
            product_details.update({'price' : values3})

            product_details.update ({'product_name': product_name})
    except:
        pass
    product_data_df = product_data_df.append(product_details, ignore_index=True)
    print(ur)

In [ ]:
# save product_data_df as csv file
with open('product_data.csv', 'a') as f:
    product_data_df.to_csv(f, mode='a', header=f.tell()==0) 
# save ingredients_df as csv file
with open('ingredients_data.csv', 'a') as f:
    ingredients_df.to_csv(f, mode='a', header=f.tell()==0) 

In [ ]:

## GET NUTRITION WAITROSE DATA

    nu_wait = soup.find(('div', 'article'))
    for po in nu_wait:
        x = po.find_all('p')
        # print(x)
        for txt in x[5: 24]:
            nutrition_waitrose_data.append(txt.text)
    nutrition_waitrose_df = pd.DataFrame( nutrition_waitrose_data).T

In [103]:
driver.quit()

## Exploratory data analysis 
